#Лабораторная работа №5
В данной работе рассматривается задача классификации изображений. Для сокращения размерности признаков, работа с изображениями осуществляется при помощи окон свёртки. Каскад свёрточных блоков в сочетании с прореживаем дает на выходе одномерное пространство признаков (Обычно, небольшой размерности). Далее, с этим пространством можно работать как с простым перцептроном, как в предыдущей работе


## Содержание работы
1. Требования к среде
2. Подготовка данных
3. Построение модели
4. Обучение и тестирование
5. Анализ результатов
6. Обучение на сущесвующих архитектурах

## Требования к среде

Для работы потребуется Python версии 3.8 и выше
Keras и Tensorflow

`pip install tensoflow, keras, opencv-python, sklearn, tqdm`


## Подготовка данных
Для эксперимента мы будем использовать классический набор данных cats/dogs. Скачать архив можно по ссылке: https://www.kaggle.com/c/dogs-vs-cats/code

В среде Google Colab у нас имеется возможность доступа к наборам данных через API. Для этого [сгенерируйте](https://www.kaggle.com/settings) (или возьмите у преподавателя) файл kaggle.json и положите его в каталог /root/.kaggle/

Теперь датасет можно загрузить всего одной строкой. Еще 3 строки потребуются, чтобы его извечь из архива


In [ ]:
!kaggle competitions download -c dogs-vs-cats
!7z x dogs-vs-cats.zip
!7z x train.zip
!7z x test1.zip

Теперь поговорим о чтении данных. Сформируем 2 массива: входные и выходные данные

In [ ]:
import os, tqdm, cv2, sklearn
import numpy as np

# Массивы данных
X = []
Y = []

# Размер изображения на входе
size = 96

# Количество классов
classes = 2

# Режим цвета
color_mode = cv2.IMREAD_COLOR #COLOR GRAYSCALE

# Прочитаем все файлы из каталога
cats_files = [f for f in os.listdir('/content/train') if f.endswith(('.png', '.jpg', '.jpeg', '.gif')) and f.startswith(('cat')) ]
dog_files = [f for f in os.listdir('/content/train') if f.endswith(('.png', '.jpg', '.jpeg', '.gif')) and f.startswith(('dog')) ]

for file in tqdm.tqdm(cats_files):
  img = cv2.resize(cv2.imread(os.path.join('/content/train', file), color_mode), (size, size))
  X.append(list(np.array(img)))
  Y.append([0])

for file in tqdm.tqdm(dog_files):
  img = cv2.resize(cv2.imread(os.path.join('/content/train', file), color_mode), (size, size))
  X.append(list(np.array(img)))
  Y.append([1])


# Преобразуем данные к Numpy-массиву
X = np.array(X)
Y = np.array(Y)

# и перемешаем
X, Y = sklearn.utils.shuffle(X, Y)

In [ ]:
# Проверить данные можно при помощи
X[60]
#Y[60]

In [ ]:
# Данный код поможет преобразовать массив с индексами классов в формат one-hot:
# так как нейронная сеть должна возратить не номер класса, а вероятности, то превратим одномерный массив индексов классов в двумерный по следующей схеме:

# [0] => [1, 0, ..., 0]
# [1] => [0, 1, ..., 0]
# ...
# [n] => [0, 0, ..., 1]
from keras.utils import to_categorical
import tensorflow as tf
Y_en = tf.keras.utils.to_categorical(Y, num_classes=2, dtype='float32')

## Построение модели

Я предлагаю исследовать разные виды моделей и снять основные их характеристики. Далее представлена архитектура свёрточной сети [96, 96, 3] в [1]. Её можно модифицировать для получения наилучших результатов.

In [ ]:
import keras
from keras.optimizers import *
from keras.models import Model, Sequential
from keras.layers import *

# Конфигурируем модель сети
def build_model(size, classes):
    # Модель последовательная...
    model = Sequential()

    # размером SIZE*SIZE*3. В первом блоке свёртка окном 3*3 разных окон 36 штук...
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(size, size, 3)))
    # Активация ReLU
    model.add(Activation("elu"))
    # Подвыборка квадратом 3*3 максимума из квадрата
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.15))

    # [32, size/3, size/3]
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    # [32, size/6, size/6]
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    # [64, size/12, size/12]
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    # [128, size/24, size/24]
    model.add(Conv2D(256, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))

    # [64, size/48, size/48]
    model.add(Conv2D(512, (3, 3), padding="same"))
    model.add(Activation("elu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # [32, size/96, size/96]
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("elu"))

    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax")) # Если будет несколько классов, то softmax
    #model.add(Activation("sigmoid"))

    # возвращаем модель
    return model

In [ ]:
# Модель можно взять уже готовую, а также сравнить её результаты с нашими.
#model = tf.keras.applications.MobileNetV2(input_shape=(96,96,3), classes = 2, weights = None)
model = tf.keras.applications.InceptionV3(input_shape=(256,256,3), classes = 2, weights = None)
#model = build_model(size, classes)
model.summary()

In [ ]:
# Строим модель и запускаем обучение
model = build_model(size, classes)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[ 'accuracy', 'mse', 'mae'])

In [ ]:
history = model.fit(X,Y_en,epochs=40, batch_size=256, validation_split=0.2)

In [ ]:
# Визуализация графика
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Функция потерь')
plt.ylabel('Потери (меньше - лучше)')
plt.xlabel('Число эпох')
plt.legend(['Обучающая', 'Тестовая'], loc='upper left')
plt.show()

В рамках группы проведите исследование параметров, влияющих на сходимость и переобучение. Для исследования возьмите следующие параметры:
1. Вид активационной функции (linear, relu, elu, silu, leaky_relu, sigmoid)
2. Величина Dropout (10%, 25%, 50%, 75%, 90%)
3. Количество свёрток на разных слоях
4. Разные виды оптимизаторов (Adam, SGD, RMSProp)
5. Разный размер batch_size (1, 8, 32, 128)
6. Разный размер входного изображения
7. Разное число каналов (RGB, HSV, Grayscale)

В качестве анализируемых величин выберем (после 10 эпох):
1. На какой эпохе accuray стал выше 90% на train и на val
2. Какое наименьшее значение loss фиксировалось на train и на val
3. Время просчёта одного изображения

На домашнее задание:
4. Построить графики loss_train, loss_val для своих кейсов
5. Определить на какой эпохе началось явное переобучение, и начиналось ли оно вовсе для выбранных кейсов

## Тестирование сети

In [ ]:
path = '/content/185.jpg'
img = cv2.resize(cv2.imread(path, color_mode), (size, size))
img = np.array([img])
pred = model.predict(img)#[list(np.array(img))])

print(pred)